Algorithm for quaternion algebra isomorphism

In [1]:
def isomorphism_same_j(B1, B2):
    """
    Defines an isomorphism of quaternion algebras,
    see Lemma 10 of "Deuring for the People: Supersingular Elliptic Curves with Prescribed Endomorphism Ring in General Characteristic".
    """
    if B1 == B2:
        return B2(1), B2(1)
    i1, _, _ = B1.gens()
    i2, j2, _ = B2.gens()
    B1_q, B1_p = B1.invariants()
    B2_q, _ = B2.invariants()
    r = B1_q/B2_q
    rnew = numerator(r).squarefree_part() / denominator(r).squarefree_part()
    rt = sqrt(r / rnew)
    bqf = DiagonalQuadraticForm(QQ, [1, -B1_p])
    x, y = bqf.solve(rnew)
    x, y = x*rt, y*rt
    gamma = x + j2*y
    new_i = i2 * gamma
    new_j = j2
    new_k = new_i * new_j
    iso = B1.hom([new_i, new_j, new_k], B2, check=False)
    return iso

def change_a(B, gamma):
    """
    Result of Albert that given isomorphic (a1, b1) (gamma, b2) we can find isomorphism between (a1, b1) and (gamma, delta) for some rational delta
    """
    alpha, beta = B.invariants()
    prod = alpha * beta
    zeta1, zeta2, zeta3 = DiagonalQuadraticForm(QQ, [alpha, beta, -prod]).solve(gamma)

    a_new = -prod*(zeta2^2 - zeta3^2 * alpha)
    b_new = gamma
    Bnew = QuaternionAlgebra(a_new, b_new)

    i, j, k = B.gens()
    new_j = zeta1*i + zeta2*j + zeta3*k
    new_i = alpha*zeta3*j + zeta2*k
    new_k = new_i * new_j

    iso = Bnew.hom([new_i, new_j, new_k], B, check=False)
    return iso

def quaternion_algebra_isomorphism(B1, B2):
    """
    Combines the above do to isomorphism
    """
    _, gamma = B1.invariants()
    iso2 = change_a(B2, gamma)
    Bnew = iso2.domain()
    iso1 = isomorphism_same_j(B1, Bnew)
    iso = iso1.post_compose(iso2)
    return iso

Testing on a small example

In [2]:
B1 = QuaternionAlgebra(-46, -87)
B2 = QuaternionAlgebra(-58, -69)

iso = quaternion_algebra_isomorphism(B1, B2)

# Check that the isomorphism preserves norms and traces
elt = B1.random_element()
elt2 = iso(elt)
assert(elt2 in B2)
assert(elt.reduced_norm() == elt2.reduced_norm())
assert(elt.reduced_trace() == elt2.reduced_trace())

For testing one may use the following function to generate pairs of isomorphic quaternion algebras.

In [5]:
def random_subset(arr):
    """Returns random subset of given array"""
    return [a for a in arr if randrange(0, 2) == 0]

def find_interesting_large_isomorphic_quaternion_algebras(pos_def=False):
    """
    Returns a pair of large isomorphic quaternion algebras where the isomorphism is unlikely to be immediatley obvious.
    May have to run it a few times to ensure they are actually large.
    """
    while True:
        s1, s2 = -1, -1
        if not pos_def:
            s1, s2 = [1, -1][randrange(0, 2)], [1, -1][randrange(0, 2)]
        a1 = s1 * randrange(2^40, 2^80)
        b1 = s2 * randrange(2^40, 2^80)
        if a1.is_prime():
            continue
        if b1.is_prime():
            continue
        B = QuaternionAlgebra(a1, b1)
        rps = B.ramified_primes()
        a2set = random_subset(rps)
        s3, s4 = -1, -1
        if not pos_def:
            s3, s4 = [1, -1][randrange(0, 2)], [1, -1][randrange(0, 2)]
        a2 = s3 * product(a2set)
        b2 = s4 * product([a for a in rps if a not in a2set])
        B2 = QuaternionAlgebra(a2, b2)
        if B.is_isomorphic(B2):
            return (B, B2)

find_interesting_large_isomorphic_quaternion_algebras(pos_def=True)

(Quaternion Algebra (-616005999200080235630071, -508645447705892759946395) with base ring Rational Field,
 Quaternion Algebra (-244770367688411513894929, -14309901468726762131) with base ring Rational Field)